In [1]:
import os
import gym
import gym_donkeycar
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2 as cv
import time
import pickle
import birds_eye_vector_space
import random
from pandas import Series, DataFrame
from collections import deque

#from keras.layers import Dense
#from tensorflow.keras.optimizers import Adam
#from keras.initializers import normal, identity
#from keras.models import model_from_json
#from keras.models import Sequential
#from keras.layers.core import Dense, Dropout, Activation, Flatten
#from keras.layers.convolutional import Convolution2D, MaxPooling2D

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.client import device_lib
#from keras import backend as K
#import tensorflow.keras.backend as K

C:\Users\studwilksa2535\AppData\Roaming\Python\Python38\site-packages\tensorflow\core\framework\tensor_shape_pb2.py:18: DeprecationWarning: Call to deprecated create function FileDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  DESCRIPTOR = _descriptor.FileDescriptor(
C:\Users\studwilksa2535\AppData\Roaming\Python\Python38\site-packages\tensorflow\core\framework\tensor_shape_pb2.py:36: DeprecationWarning: Call to deprecated create function FieldDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  _descriptor.FieldDescriptor(
C:\Users\studwilksa2535\AppData\Roaming\Python\Python38\site-packages\tensorflow\core\framework\tensor_shape_pb2.py:29: DeprecationWarning: Call to deprecated create function Descriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find d

In [2]:
# use gpu
#K.tensorflow_backend._get_available_gpus()
#sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
#K.tensorflow_backend._get_available_gpus()

In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  2


In [4]:
tf.test.is_built_with_cuda()

True

In [5]:
tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [6]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]

In [7]:
# run on the second GPU
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
#os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [8]:
len(tf.config.list_physical_devices('GPU'))
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 2481959148458637441
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 9401860096
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 7129554413132180214
 physical_device_desc: "device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:49:00.0, compute capability: 7.5"
 xla_global_id: 416903419,
 name: "/device:GPU:1"
 device_type: "GPU"
 memory_limit: 6271991808
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 13023213272160920217
 physical_device_desc: "device: 1, name: GeForce RTX 2070 SUPER, pci bus id: 0000:21:00.0, compute capability: 7.5"
 xla_global_id: 2144165316]

In [9]:
#mirrored_strategy = tf.distribute.MirroredStrategy()

In [10]:
#sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [11]:
# Supress scientific notation like: e*+03
np.set_printoptions(suppress=True)

In [12]:
ANCHOR_POINTS = 8
ROAD_ROI = np.array([(120,90),(200,90),(0,200),(320,200)],dtype='float32')
WARPED_IMAGE_SHAPE = np.array([[10,320],[0,0],[200,0],[200,310]],np.int32)            # NEW IMAGE Shape after Warping !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
WARPED_IMAGE_HEIGHT = 320
WARPED_IMAGE_WIDTH = 200
NUMBER_OF_DEPTH_LAYERS = 50

VECTOR_SPACE_IMAGE_ROWS = 50
VECTOR_SPACE_IMAGE_COLUMNS = 100
VECTOR_SPACE_IMAGE_CHANNELS = 4 # 4*(50,100) stacked frames 

In [13]:
# function to save the plots
def save_plot(data, name, episode):
    plt.figure(figsize=(8,5), frameon=True)
    #plt.plot([episode for episode in range(len(data))], data)
    plt.plot([ep for ep in range(episode)], data)
    plt.xlabel('Episodes')
    plt.ylabel(name)
    figplot = '%s-episode-%s.png' % (name,episode)
    #figplot = '{}-episode-{}.png'.format(name,episode)
    #location = '{}/saves/torch/{}'.format(os.getcwd(),figplot)
    #location = '%s/collected_data/plots/%s' % (os.getcwd(),figplot)
    location = 'C:\\Users\\studwilksa2535\\Desktop\\DonkeyCarAI\\collected_data\\plots\\%s' % figplot 
    plt.savefig(location, transparent=False)
    plt.close()

In [14]:
def save_data_as_dataframe(episode_number, reward, loss, measured_time, max_cte, average_cte, last_lap_time):
    episodes = [i for i in range(1, episode_number+1)]
    collected_data = {'episode': episodes, 'reward': reward, 'loss':loss, 'time': measured_time, 'max_cte': max_cte, 'average_cte': average_cte, 'lap_time': last_lap_time}
    df_data = DataFrame.from_dict(collected_data).set_index('episode')
    
    df_name = 'data-episode-%s.pkl' % episode
    #location = '%s/collected_data/raw_data/%s' % (os.getcwd(),df_name)
    location = 'C:\\Users\\studwilksa2535\\Desktop\\DonkeyCarAI\\collected_data\\raw_data\\%s' % df_name
    df_data.to_pickle(location)

In [15]:
vector_space = birds_eye_vector_space.Vector_space(ANCHOR_POINTS, ROAD_ROI, WARPED_IMAGE_SHAPE, WARPED_IMAGE_HEIGHT, WARPED_IMAGE_WIDTH, NUMBER_OF_DEPTH_LAYERS)

vector space is initialized
loading camera parameters


In [16]:
os.getcwd()

'C:\\Users\\studwilksa2535\\Desktop\\DonkeyCarAI\\Master-Thesis-Development-of-a-Deep-RL-Model-for-simulated-Driving-2D-Vector-Space\\docker\\src'

In [17]:
class DQNAgent:
    
    def __init__(self, state_size, action_size):
        self.t = 0
        self.max_Q = 0
        self.train = True
        # Set to True to train on images with segmented lane lines
        self.lane_detection = False
        
        # Huber loss
        #self.huber_loss = tf.keras.losses.Huber(reduction=tf.keras.losses.Reduction.SUM)
        
        # Get size of state and action
        self.state_size = state_size
        self.action_size = action_size
        
        # CNN Input
        #self.input_shape = (VECTOR_SPACE_IMAGE_CHANNELS, VECTOR_SPACE_IMAGE_ROWS, VECTOR_SPACE_IMAGE_COLUMNS, 4)   # == 4 * (50, 100, 1); keras: (1,50,100,4)
        self.input_shape = (1, VECTOR_SPACE_IMAGE_ROWS, VECTOR_SPACE_IMAGE_COLUMNS, VECTOR_SPACE_IMAGE_CHANNELS)   # == 4 * (50, 100, 1); keras: (1,50,100,4)
        
        # These are hyper parameters for the DQN
        self.discount_factor = 0.99
        self.learning_rate = 1e-4
        if (self.train):
            self.epsilon = 1.0
            self.initial_epsilon = 1.0
        else:
            self.epsilon = 1e-6
            self.initial_epsilon = 1e-6
        self.epsilon_min = 0.02
        self.batch_size = 512
        self.train_start = 100
        self.explore = 10000
        
        # Create replay memory using deque
        self.memory = deque(maxlen=10000)
        
        # Create main model and target model                # Double DQN !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        self.model = self.build_model()
        self.target_model = self.build_model()

        # Copy the model to target model
        # --> initialize the target model so that the parameters of model & target model to be same
        self.update_target_model()
        
    def build_model_old(self):
        print('delte this')
        #model = Sequential()
        #model.add(Convolution2D(32, 8, 8, subsample=(4, 4), border_mode='same',input_shape=(VECTOR_SPACE_IMAGE_ROWS,VECTOR_SPACE_IMAGE_COLUMNS,VECTOR_SPACE_IMAGE_CHANNELS)))  #80*80*4
        #model.add(Activation('relu'))
        #model.add(Convolution2D(64, 4, 4, subsample=(2, 2), border_mode='same'))
        #model.add(Activation('relu'))
        #model.add(Convolution2D(64, 3, 3, subsample=(1, 1), border_mode='same'))
        #model.add(Activation('relu'))
        #model.add(Flatten())
        #model.add(Dense(512))
        #model.add(Activation('relu'))

        ## 15 categorical bins for Steering angles
        #model.add(Dense(15, activation="linear")) 

        #adam = Adam(lr=self.learning_rate)
        #model.compile(loss='mse',optimizer=adam)

        #return model

    def build_model(self):
        model = keras.Sequential([
            layers.Conv2D(filters=32, strides=(4, 4),kernel_size=8, padding='same', activation='relu', input_shape=(self.input_shape[1:])),
            layers.Conv2D(filters=64, strides=(2, 2),kernel_size=4, padding='same', activation='relu'),
            layers.Conv2D(filters=64, strides=(1, 1),kernel_size=3, padding='same', activation='relu'),
            layers.Flatten(),
            layers.Dense(units=512,activation='relu'),
            layers.Dense(units=512,activation='relu'), # TODO ADDED THIS LAYER FOR TESTING
            layers.Dense(units=15,activation='linear'),
        ])
        #optimizer = tf.keras.optimizers.Adam(learning_rate=self.learning_rate)
        optimizer = tf.keras.optimizers.Adam()                                  # TODO UNCOMMENT THIS
        #optimizer = tf.keras.optimizers.SGD()
        model.compile(loss='mse', optimizer=optimizer)                          # TODO UNCOMMENT THIS
        #model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=optimizer)
        return model
    
    def calculate_loss(self, y_true, y_pred):    
        y_pred = tf.convert_to_tensor_v2(y_pred)
        y_true = tf.cast(y_true, y_pred.dtype)
        return tf.reduce_mean(math_ops.square(y_pred - y_true), axis=-1)
    
    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

    # Get action from model using epsilon-greedy policy
    def get_action(self, s_t):
        if np.random.rand() <= self.epsilon:
            #print("Return Random Value")
            #return random.randrange(self.action_size)
            return np.random.uniform(-1,1)
        else:
            #print("Return Max Q Prediction")
            q_value = self.model.predict(s_t)
            # Convert q array to steering value
            return linear_unbin(q_value[0])

    def replay_memory(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        #if self.epsilon > self.epsilon_min:
        #    #self.epsilon *= self.epsilon_decay
        #    self.epsilon -= (self.initial_epsilon - self.epsilon_min) / self.explore

    def train_replay(self):
        if len(self.memory) < self.train_start:
            return
        
        batch_size = min(self.batch_size, len(self.memory))
        minibatch = random.sample(self.memory, batch_size)

        state_t, action_t, reward_t, state_t1, terminal = zip(*minibatch)
        #print('Starting training\nstate_t: %s\naction_t: %s\nreward_t: %s\nstate_1: %s\nterminal: %s' % (len(state_t), len(action_t), len(reward_t), len(state_t1), len(terminal)))
        state_t = np.concatenate(state_t)
        state_t1 = np.concatenate(state_t1)
        #print('Concat:\nstate: %s\nstate_t1: %s' % (state_t, state_t1))
        targets = self.model.predict(state_t)
        self.max_Q = np.max(targets[0])
        target_val = self.model.predict(state_t1)
        target_val_ = self.target_model.predict(state_t1)
        for i in range(batch_size):
            if terminal[i]:
                targets[i][action_t[i]] = reward_t[i]
            else:
                a = np.argmax(target_val[i])
                targets[i][action_t[i]] = reward_t[i] + self.discount_factor * (target_val_[i][a])

        
        # train on the current batch and return a dictionary with the loss and so on
        #metrics = self.model.train_on_batch(x=state_t, y=targets, reset_metrics=True, return_dict=True)           
        metrics = self.model.train_on_batch(x=state_t, y=targets, reset_metrics=True, return_dict=True) #reset Metrix different
        return metrics['loss']
        
    def load_model(self, name):
        self.model.load_weights(name)

    # Save the model which is under training
    def save_model(self, name):
        self.model.save_weights(name)

In [18]:
def linear_bin(a):
    """
    Convert a value to a categorical array.

    Parameters
    ----------
    a : int or float
        A value between -1 and 1

    Returns
    -------
    list of int
        A list of length 15 with one item set to 1, which represents the linear value, and all other items set to 0.
    """
    a = a + 1
    b = round(a / (2 / 14))
    arr = np.zeros(15)
    arr[int(b)] = 1
    return arr

In [19]:
def linear_unbin(arr):
    """
    Convert a categorical array to value.

    See Also
    --------
    linear_bin
    """
    if not len(arr) == 15:
        raise ValueError('Illegal array length, must be 15')
    b = np.argmax(arr)
    a = b * (2 / 14) - 1
    return a

## Main training loop

In [ ]:
#%% SET UP ENVIRONMENT
# get same weights at the model weihgts init
tf.random.set_seed(42)

# Normal one 
#os.environ['DONKEY_SIM_PATH'] = "/home/zamy/masterthesis/DonkeySimLinux/donkey_sim.x86_64"
os.environ['DONKEY_SIM_PATH'] = "C:\\Users\\studwilksa2535\\Desktop\\DonkeyCarAI\\test\\DonkeySimWin\\donkey_sim.exe"
os.environ['DONKEY_SIM_PORT'] = str(9091)
os.environ['DONKEY_SIM_HEADLESS'] = str(0) # "1" is headless

CAMERA_CONF = {'cam_config':{'img_w': '320',
                        'img_h': '200',
                        'img_d': '1',   # 3 for colored Tensor image
                        'img_enc': 'PNG', 
                        'fov': '90', 
                        'fish_eye_x': '0.0', 
                        'fish_eye_y': '0.0', 
                        'offset_x': '0.0', 
                        'offset_y': '0.0', 
                        'offset_z': '0.0', 
                        'rot_x': '0'}}
# Other tracks
#env = gym.make("donkey-generated-track-v0",conf=CAMERA_CONF)

# TRACKS TRACKS - TKarmer Tracks
#env = gym.make("donkey-generated-track-v0",conf=CAMERA_CONF) #,conf=config
# Random track, but the reward is kinda not working, IF THE REWARD WORKS IT IS KINDA RANDOM !!, this maybe debends on the environment
env = gym.make("donkey-generated-roads-v0",conf=CAMERA_CONF) #,conf=config

# Mini Monaco Track
#env = gym.make("donkey-minimonaco-track-v0",conf=CAMERA_CONF) #,conf=config
#env = gym.make("donkey-generated-track-v0") #,conf=config
#env = gym.wrappers.ResizeObservation(env,(200,320))

## DELETE LATER
#env.frameskip = 1
#gym.wrappers.max

# Create DQN Model
# Get size of state and action from environment
state_size = (VECTOR_SPACE_IMAGE_ROWS, VECTOR_SPACE_IMAGE_COLUMNS, VECTOR_SPACE_IMAGE_CHANNELS)
action_size = 15 # Steering and Throttle
throttle = 0.15 # Set the throttle as a constant value
agent = DQNAgent(state_size, action_size)


# Arrays for data collection / exploratory data analysis
average_cte = []
average_speed = []
measured_time = []
average_rewards = []
average_loss = []
collected_max_cte = []
collected_lap_time = []

# when a model and the data should be saved
save_state = 200

for episode in range(1,7001):
    # Resetting the environment and preprocessing the first image
    observation = env.reset()
    # Graykonvertion of the observation image
    observation = cv.cvtColor(observation, cv.COLOR_BGR2GRAY)
    observation = vector_space.image_preprocessing(observation)
    # frame stacking (4 times)
    obv_stack = np.stack((observation, observation, observation, observation), axis= 2)
    # reshaping for keras
    obv_stack = obv_stack.reshape(1, obv_stack.shape[0], obv_stack.shape[1], obv_stack.shape[2])
    
    #action = np.array([0,0.10]) # drive straight with small speed
    #action = np.array([1,0.1]) # drive straight with small speed
    
    # summed up values for data collection
    # cumulative values for data collection
    total_cte = float(0)
    total_speed = float(0)
    total_reward = float(0)
    total_time = time.time()
    total_loss = float(0)
    max_cte = int(0)
    lap_time = int(0)
        
    # counting the amount of frames per episode
    frames = int(0)
    
    # boolean that describes if the env is done with this episode
    done = False
    while not done:
        # incrementing the amount of frames per episode
        frames += 1
        
        # Making a prediction for the current state
        # and getting the information form the next step
        steering = agent.get_action(obv_stack)
        action = [steering, throttle]
        next_observation, reward, done, info = env.step(action)
        
        current_cte = abs(info['cte'])
        # punish and reward for a low or high 'CTE' with the logarithmic function
        if current_cte < .25:
            reward += abs(np.log(.10)) * 8  # 4 before
        elif current_cte >= .25 and current_cte < .5:
            reward += current_cte * 2
        elif current_cte >= .5 and current_cte < 1: 
            reward += abs(np.log(current_cte))
        elif current_cte >= 1 and current_cte < 2: 
            reward -= abs(current_cte) * 2
        elif current_cte >= 2 and current_cte < 3: 
            reward -= abs(current_cte) * 3
        elif current_cte >= 3 and current_cte < 4: 
            reward -= abs(current_cte) * 4
        elif current_cte >=4:
            reward -= abs(current_cte) * 5
                    
        # if the car leaves the track punish it
        if done:
            reward -= 100
        # Check if the CTE of the environment is bugged
        if episode == 1:
            #print('CTE: %s\tReward: %s' % (info['cte'], reward)
            print('REWARD: %s\tCTE: %s' % (reward, current_cte))
        
        # Graykonvertion of the observation image
        next_observation = cv.cvtColor(next_observation, cv.COLOR_BGR2GRAY)
        
        # preprocessing the new observation
        next_observation_show = vector_space.image_preprocessing(next_observation)
        # reshaping for keras
        #next_observation = next_observation_show.reshape(1, next_observation_show.shape[0], next_observation_show.shape[1], 1)
        # appending to the observation stack
        #obv_stack = np.append(next_observation, obv_stack[:, :, :, :3], axis=3)
        
        obv_stack_t1 = np.stack([next_observation_show, obv_stack[0,:,:,0],obv_stack[0,:,:,1],obv_stack[0,:,:,2]], axis=2)
        obv_stack_t1 = obv_stack_t1.reshape(1, obv_stack_t1.shape[0], obv_stack_t1.shape[1], obv_stack_t1.shape[2])
                
        # saving the sample <s, a, r, s'> to the replay memory
        agent.replay_memory(obv_stack, np.argmax(linear_bin(steering)), reward, obv_stack_t1, done)        
        
        # training the DDQN, if training is enabled
        #if agent.train:
        #    loss = agent.train_replay()
        #    if loss != None:
        #        total_loss += loss
        #        #print('loss: %s ' % (total_loss / frames))
        
        # overwriting the stack and incrementing the time/frame counter
        obv_stack = obv_stack_t1
        agent.t += 1       
        
        # adding up the collected data
        current_cte = info['cte'] 
        total_cte += current_cte
        total_speed += info['speed']
        total_reward += reward
        # update if there is any change in incrementation 
        if max_cte < current_cte:
            max_cte = current_cte
        if lap_time < info['last_lap_time']:
            lap_time = info['last_lap_time']
            
        cv.imshow('vec img', next_observation_show)
        
        # stop the training if the car passes 120 seconds,
        # so there is no infinit learning
        if abs(total_time - time.time()) >= 120:
            done = True

        # Graykonvertion of the observation image
        #observation = cv.cvtColor(observation, cv.COLOR_BGR2GRAY)
        
        # Image preprocessing
        #vec_image = vector_space.image_preprocessing(observation)
        
        #print('REWARD: %s' % reward)
        
        #cv.imshow('vec img', vec_image)
        
        #break       # TODO: DELTE LATER
        if cv.waitKey(25) & 0xFF == ord('q'):
            cv.destroyAllWindows()
            break
    
    # Training after each episode
    # do 10 training sets of each with a batch size of 512
    for i in range(0,10):
        if agent.train:
            total_loss += agent.train_replay()
            #total_loss = agent.train_replay()
            #if loss != None:
            #    total_loss = loss
    
    
    # AFTER this Episode & the environment returns True for the done variable
    # updating the target DDQN
    agent.update_target_model()
    
    # calculating the average loss in this episode
    #episode_loss = total_loss.copy() #/ frames
    total_loss = total_loss / 512 * 10
    episode_cte = total_cte / frames
    episode_reward = total_reward / frames
    
    # updating the time parameter
    total_time = abs(total_time - time.time())
    
    # appending the collected data
    average_cte.append(episode_cte)
    average_speed.append(total_speed / frames)
    measured_time.append(total_time)
    average_rewards.append(episode_reward)
    average_loss.append(total_loss)
    collected_max_cte.append(max_cte)
    collected_lap_time.append(lap_time) 
    
    # Testing of saving the plot
    #if episode % 3 == 0 and episode != 0:
    #    save_plot(colleted_rewards, 'Reward', episode)
    #    cv.destroyAllWindows()
    #    break
    
    # Print episode information    
    print('EPISODE: %s | TIME: %s s | REWARD: %s | FRAMES: %s | QMAX: %s | EPSILON: %s | CTE: %s | LOSS: %s' % (episode, total_time, episode_reward, frames, str(agent.max_Q), agent.epsilon, episode_cte, total_loss))    
    
    # update the epsilon after each episode
    # Epsilion decay over time/amount of trainings
    if agent.epsilon >= agent.epsilon_min:
        #self.epsilon *= self.epsilon_decay
        #self.epsilon -= (self.initial_epsilon - self.epsilon_min) / self.explore
        #self.epsilon -= 0.00025
        agent.epsilon -= 0.00025 # before 0.00035
    
    if episode % save_state == 0 and episode != 0:
        print('saving after %s episodes' % save_state)
        # saving the model
        agent.save_model('C:\\Users\\studwilksa2535\\Desktop\\DonkeyCarAI\\models\\model_episode_%s.h5' % episode)        
        
        # Saving the data as plots
        save_plot(average_rewards, 'Reward', episode)
        save_plot(average_cte, 'Average cte', episode)
        save_plot(collected_max_cte, 'Max cte', episode)
        save_plot(measured_time, 'Time', episode)
        save_plot(average_loss, 'Loss', episode)
        save_plot(collected_lap_time, 'Lap Time', episode)
        
        # Saving the collected data
        save_data_as_dataframe(episode, average_rewards, average_loss, measured_time, collected_max_cte, average_cte, collected_lap_time)
    
    # closing all cv windows
    cv.destroyAllWindows()

# closing all cv windows
cv.destroyAllWindows()
# Close the enviroment
env.close()

INFO:gym_donkeycar.core.client:connecting to localhost:9091 
C:\Users\studwilksa2535\AppData\Roaming\Python\Python38\site-packages\gym\spaces\box.py:84: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


starting DonkeyGym env
Setting default: start_delay 5.0
Setting default: max_cte 5.0
Setting default: frame_skip 1
Setting default: cam_resolution (120, 160, 3)
Setting default: log_level 20
Setting default: host localhost
Setting default: port 9091
loading scene generated_road


INFO:gym_donkeycar.envs.donkey_sim:on need car config
INFO:gym_donkeycar.envs.donkey_sim:sending car config.
INFO:gym_donkeycar.envs.donkey_sim:done sending cam config. {'img_w': '320', 'img_h': '200', 'img_d': '1', 'img_enc': 'PNG', 'fov': '90', 'fish_eye_x': '0.0', 'fish_eye_y': '0.0', 'offset_x': '0.0', 'offset_y': '0.0', 'offset_z': '0.0', 'rot_x': '0'}


REWARD: 18.420681614844284	CTE: 0.0001089728
REWARD: 18.421490381413086	CTE: 0.0001163412
REWARD: 18.521874202759417	CTE: 8.60314e-05
REWARD: 18.58218235154151	CTE: 6.78749e-05
REWARD: 18.66266220265344	CTE: 0.0005586849
REWARD: 18.73641477964156	CTE: 0.001514492
REWARD: 18.7993648877511	CTE: 0.0009789383
REWARD: 18.872165488799563	CTE: 0.001183095
REWARD: 18.931727204442655	CTE: 0.0004494247
REWARD: 18.99100903644468	CTE: 0.001171923
REWARD: 19.05821925281783	CTE: 0.003892349
REWARD: 19.123453394728294	CTE: 0.008412085
REWARD: 19.19278667449038	CTE: 0.01076267
REWARD: 19.248984158827437	CTE: 0.01097055
REWARD: 19.306526305934547	CTE: 0.01391135
REWARD: 19.349989517162268	CTE: 0.01952473
REWARD: 19.407504091910226	CTE: 0.01856712
REWARD: 19.452962632277323	CTE: 0.01755981
REWARD: 19.503256136729036	CTE: 0.02377668
REWARD: 19.539366269099467	CTE: 0.03218872
REWARD: 19.577130262268213	CTE: 0.03254003
REWARD: 19.63241557195433	CTE: 0.03638801
REWARD: 19.686769455060823	CTE: 0.0386857
REWA

REWARD: 3.7589602133183613	CTE: 0.8344843
REWARD: 3.776037326515146	CTE: 0.8332469
REWARD: 3.806942389561377	CTE: 0.8205779
REWARD: 3.8589326011614067	CTE: 0.8006896
REWARD: 3.8746352747957644	CTE: 0.7799706
REWARD: 3.911964514215298	CTE: 0.7724415
REWARD: 3.942871394862798	CTE: 0.769747
REWARD: 3.9589957111091616	CTE: 0.7730636
REWARD: 3.9791382739406904	CTE: 0.7770313
REWARD: 4.001316830734375	CTE: 0.7610068
REWARD: 3.947072246189693	CTE: 0.792816
REWARD: 3.8671430630208645	CTE: 0.8175608
REWARD: 3.805370603529482	CTE: 0.8447918
REWARD: 3.7453896048508275	CTE: 0.858537
REWARD: 3.7580829353547895	CTE: 0.8603801
REWARD: 3.7880824967008047	CTE: 0.8530317
REWARD: 3.815336870006453	CTE: 0.8509289
REWARD: 3.8387707990587914	CTE: 0.8447143
REWARD: 3.9389560595454545	CTE: 0.810507
REWARD: 3.9551848767410758	CTE: 0.8071093
REWARD: 3.984430697399465	CTE: 0.7893094
REWARD: 4.068147664736581	CTE: 0.7636992
REWARD: 4.044120160907637	CTE: 0.7497497
REWARD: 4.041114152143383	CTE: 0.7549401
REWARD: 

...

EPISODE: 5524 | TIME: 34.26650547981262 s | REWARD: -4.2458289238904285 | FRAMES: 179 | QMAX: 802.1822 | EPSILON: 0.019750000000082396 | CTE: -1.790059853233519 | LOSS: 21.95227161049843
EPISODE: 5525 | TIME: 127.96787285804749 s | REWARD: 9.51684904588904 | FRAMES: 809 | QMAX: 934.59955 | EPSILON: 0.019750000000082396 | CTE: -0.6220002293980226 | LOSS: 26.85937225818634
EPISODE: 5526 | TIME: 104.83795166015625 s | REWARD: -2.8467639862816903 | FRAMES: 655 | QMAX: 649.5125 | EPSILON: 0.019750000000082396 | CTE: -2.022136036836748 | LOSS: 21.79399222135544
EPISODE: 5527 | TIME: 98.26132941246033 s | REWARD: 5.19990047973227 | FRAMES: 606 | QMAX: 808.90344 | EPSILON: 0.019750000000082396 | CTE: -0.9128650023899338 | LOSS: 24.566414654254913
EPISODE: 5528 | TIME: 36.005695819854736 s | REWARD: 3.9628287330823397 | FRAMES: 190 | QMAX: 911.258 | EPSILON: 0.019750000000082396 | CTE: -1.2875222379694735 | LOSS: 24.921793043613434
EPISODE: 5529 | TIME: 128.0708248615265 s | REWARD: 7.111233274

EPISODE: 5568 | TIME: 55.583449602127075 s | REWARD: 16.219189471301792 | FRAMES: 321 | QMAX: 939.1669 | EPSILON: 0.019750000000082396 | CTE: -0.14679654635202488 | LOSS: 25.80663725733757
EPISODE: 5569 | TIME: 56.342437505722046 s | REWARD: -2.4602347151134447 | FRAMES: 324 | QMAX: 963.6978 | EPSILON: 0.019750000000082396 | CTE: -1.8461169800330237 | LOSS: 23.88748601078987
EPISODE: 5570 | TIME: 111.83492493629456 s | REWARD: 12.457652276110927 | FRAMES: 703 | QMAX: 730.6572 | EPSILON: 0.019750000000082396 | CTE: -0.3832500641450641 | LOSS: 28.489902913570404
EPISODE: 5571 | TIME: 128.02379155158997 s | REWARD: 13.271017626016787 | FRAMES: 805 | QMAX: 871.15906 | EPSILON: 0.019750000000082396 | CTE: 0.12032693611055903 | LOSS: 26.525350362062454
EPISODE: 5572 | TIME: 122.17185926437378 s | REWARD: 0.5480375204405479 | FRAMES: 765 | QMAX: 980.096 | EPSILON: 0.019750000000082396 | CTE: -1.7059554621360773 | LOSS: 29.541032016277313
EPISODE: 5573 | TIME: 41.01775312423706 s | REWARD: 2.7

EPISODE: 5612 | TIME: 80.36471843719482 s | REWARD: 14.676732091829843 | FRAMES: 474 | QMAX: 942.6532 | EPSILON: 0.019750000000082396 | CTE: 0.026934912478481014 | LOSS: 29.377977699041367
EPISODE: 5613 | TIME: 128.21286129951477 s | REWARD: 14.110088482358789 | FRAMES: 796 | QMAX: 980.2693 | EPSILON: 0.019750000000082396 | CTE: 0.1099948121108039 | LOSS: 25.49479454755783
EPISODE: 5614 | TIME: 37.071441411972046 s | REWARD: 7.254795355542274 | FRAMES: 196 | QMAX: 940.4289 | EPSILON: 0.019750000000082396 | CTE: -0.7933746301913266 | LOSS: 24.954090267419815
EPISODE: 5615 | TIME: 128.21107053756714 s | REWARD: 6.214816581782831 | FRAMES: 804 | QMAX: 1028.6111 | EPSILON: 0.019750000000082396 | CTE: -1.0281195253824875 | LOSS: 28.35733398795128
EPISODE: 5616 | TIME: 54.58173155784607 s | REWARD: -3.2943147372096364 | FRAMES: 309 | QMAX: 926.4241 | EPSILON: 0.019750000000082396 | CTE: -2.224882866776117 | LOSS: 27.47898578643799
EPISODE: 5617 | TIME: 39.216557264328 s | REWARD: 14.80903603

EPISODE: 5656 | TIME: 128.19163131713867 s | REWARD: 14.179234444454405 | FRAMES: 800 | QMAX: 899.6883 | EPSILON: 0.019750000000082396 | CTE: 0.1699839649637001 | LOSS: 36.37610450387001
EPISODE: 5657 | TIME: 69.05768489837646 s | REWARD: -12.039507361531667 | FRAMES: 402 | QMAX: 896.33875 | EPSILON: 0.019750000000082396 | CTE: -3.378916137699003 | LOSS: 32.64866188168526
EPISODE: 5658 | TIME: 55.54652142524719 s | REWARD: 12.738934428074838 | FRAMES: 318 | QMAX: 1016.51886 | EPSILON: 0.019750000000082396 | CTE: 0.21907808017735844 | LOSS: 28.375033289194107
EPISODE: 5659 | TIME: 98.29910707473755 s | REWARD: 0.041541856591543094 | FRAMES: 609 | QMAX: 999.67944 | EPSILON: 0.019750000000082396 | CTE: -1.6780564378080451 | LOSS: 30.748158246278763
EPISODE: 5660 | TIME: 65.53990459442139 s | REWARD: -11.240777734221787 | FRAMES: 380 | QMAX: 842.1284 | EPSILON: 0.019750000000082396 | CTE: -3.348328398473952 | LOSS: 29.911947548389435
EPISODE: 5661 | TIME: 58.43166947364807 s | REWARD: 14.5

EPISODE: 5700 | TIME: 58.49046730995178 s | REWARD: -2.358373872505724 | FRAMES: 331 | QMAX: 930.3373 | EPSILON: 0.019750000000082396 | CTE: -2.0782510094773405 | LOSS: 36.82547092437744
EPISODE: 5701 | TIME: 128.2389838695526 s | REWARD: 9.629088270175282 | FRAMES: 796 | QMAX: 1042.7847 | EPSILON: 0.019750000000082396 | CTE: -0.4914276429488693 | LOSS: 33.63990664482117
EPISODE: 5702 | TIME: 88.78536343574524 s | REWARD: -4.745045814622597 | FRAMES: 534 | QMAX: 1052.9153 | EPSILON: 0.019750000000082396 | CTE: -2.4982548335283927 | LOSS: 35.32275080680847
EPISODE: 5703 | TIME: 58.123218297958374 s | REWARD: -0.7049274680778007 | FRAMES: 330 | QMAX: 1036.8813 | EPSILON: 0.019750000000082396 | CTE: -1.8390517734843927 | LOSS: 28.436098098754883
EPISODE: 5704 | TIME: 75.4610493183136 s | REWARD: 6.224788040339111 | FRAMES: 452 | QMAX: 982.6239 | EPSILON: 0.019750000000082396 | CTE: -1.0655746627926992 | LOSS: 32.752437591552734
EPISODE: 5705 | TIME: 89.62307071685791 s | REWARD: 14.865122